# 0-1 knapsack using genetic algorithms

##### Import needed libraries

In [1]:
import numpy as np
import collections

#### Define variables

In [2]:
Item = collections.namedtuple('knapsack', 'weight value')  
#namedtuple is type of container like dictionaries in the collections module
knapsack_capacity = 0
knapsack_max_weight = 0
population_size = 0
probability_of_crossover = 0.0
probability_of_mutation = 0.0
generations_number = 0


In [3]:
# Named Tuple, its values are accessible through fields names as well, e.g Individual.weight e.t.c in this case.
Individual = collections.namedtuple('population', 'chromosome weight value')


##### Generate Population

In [4]:
def generate_population(size, knapsack_capacity):
    new_population = []

    for i in range(size):
        item = np.random.randint(0, knapsack_capacity - 1)

        ### Initialize Random population
        new_population.append(
            Individual( chromosome=np.random.randint(2, size=(1, knapsack_capacity))[0], weight=-1, value=-1 )
        )

    return new_population

##### Select a parent from the generated Population

In [5]:
def parent_selection(population):
    parents = []
    total_value = 0

    for individual in population:
        total_value += individual.value

    # Find Fittest Individual to select parent
    highest, second_highest = find_two_fittest_individuals(population)
    parents.append(highest)
    parents.append(second_highest)

    # Check Total sum value of fittest individuals
    sum_value = 0
    while len(parents) < len(population):
        individual = np.random.randint(0, len(population) - 1)
        sum_value += population[individual].value

        if sum_value >= total_value:
            parents.append(population[individual])

    return parents

##### Apply Crossover & Mutation

In [6]:
def apply_crossover(population, knapsack_capacity, probability_of_crossover, probability_of_mutation):
    crossovered_population = []

    while len(crossovered_population) < len(population):
        if np.random.randint(0, 100) <= probability_of_crossover * 100:
            parent_a = np.random.randint(0, len(population) - 1)
            parent_b = np.random.randint(0, len(population) - 1)

            chromosome_a = np.concatenate((population[parent_a].chromosome[:int(knapsack_capacity / 2)],
                                        population[parent_b].chromosome[int(knapsack_capacity / 2):]))
            
            chromosome_b = np.concatenate((population[parent_a].chromosome[int(knapsack_capacity / 2):],
                                        population[parent_b].chromosome[:int(knapsack_capacity / 2)]))
            # Apply Mutation on chromosomes
            chromosome_a = apply_mutation(chromosome_a, knapsack_capacity, probability_of_mutation)

            chromosome_b = apply_mutation(chromosome_b, knapsack_capacity, probability_of_mutation)

            crossovered_population.append(Individual(chromosome=chromosome_a,weight=-1,value=-1))
            
            crossovered_population.append(Individual(chromosome=chromosome_b,weight=-1,value=-1))

    return crossovered_population

In [7]:
def calculate_weight_value(chromosome, knapsack):
    weight = 0
    value = 0

    for i, gene in enumerate(chromosome):
        if gene == 1:
            weight += knapsack[i].weight
            value += knapsack[i].value

    return weight, value

In [8]:
def apply_mutation(chromosome, knapsack_capacity, probability_of_mutation):
    if np.random.randint(0, 100) <= probability_of_mutation * 100:
        genes = np.random.randint(0, 2)

        for i in range(genes):
            gene = np.random.randint(0, knapsack_capacity - 1)
            if chromosome[gene] == 0:
                chromosome[gene] = 1
            else:
                chromosome[gene] = 0

    return chromosome

In [9]:
def get_value(chromosome) :
    return chromosome.value

In [10]:
def find_two_fittest_individuals(population):
    population.sort(key=get_value, reverse=True)
    return population[0], population[1]

In [11]:
def calculate_fitness(population, items, max_weight):
    for i in range(len(population)) :
        weight, value = calculate_weight_value(population[i].chromosome, items)
        population[i] = population[i]._replace(weight=weight, value=value)

        while (weight > max_weight) :
            apply_mutation(population[i].chromosome, knapsack_capacity, probability_of_mutation)
            weight, value = calculate_weight_value(population[i].chromosome, items)
            population[i] = population[i]._replace(weight=weight, value=value)

    return population

In [12]:
def execute_genetic_algo():
   
    population = generate_population(population_size, knapsack_capacity)
    
    print(generations_number)
    
    value = []
    iteration = []
    best_solution = None

    for i in range(generations_number):
        
        ## Calculate Fitness of initial population
        fitness = calculate_fitness(population, items, max_weight)
        
        ## Select parent
        parents = parent_selection(fitness)
        
        ## Apply crossover and mutation
        crossovered = apply_crossover(parents, knapsack_capacity, probability_of_crossover, probability_of_mutation)
        ## Calculate Fitness of population
        population = calculate_fitness(crossovered, items, max_weight)
        ## Find fittest cadidates
        candidate, _ = find_two_fittest_individuals(population)
        if best_solution is None:
            best_solution = candidate
        elif candidate.value > best_solution.value:
            best_solution = candidate

        value.append(best_solution.value)
        iteration.append(i)
        
        ### print Every 50th generation results
        if i % 50 == 0:
            print ('\nCurrent generation  : {}'.format(i))
            print ('  Best solution so far: {}'.format(best_solution.value))

    print ('\n--------------------------------')
    print (' solution found:')
    print ('\nWeight: {}'.format(best_solution.weight))
    print ('Value : {}'.format(best_solution.value))
    print ('\nThe Best Chromosome: {}'.format(best_solution.chromosome))


In [13]:
if __name__ == "__main__":
     
       
        # Initialize random population size
        population_size = np.random.randint(50, 100)    
        
        # Specify Random Number for the generations / offsring
        generations_number = np.random.randint(30, 700)
        
         #### Crossover and Mutation Probabilities
        probability_of_crossover = round(np.random.uniform(low=0.3, high=0.8), 1)
        probability_of_mutation  = round(np.random.uniform(low=0.0, high=0.5), 1)
        
        ## Initialize capacity
        print('Enter the knapsack capacity:')
        knapsack_capacity = int(input())
        
        ## Max weight
        print('Enter the maximum weight of knapsack:')
        max_weight = int(input())
        

        items = []
        for i in range(knapsack_capacity):
            print ('Enter the weight of item{}'.format(i+1))
            weight=int(input())
            print ('Enter the Value of item{}'.format(i+1))
            value=int(input())
            items.append(Item(weight, value))
       
    
        print (' Generated Parameters             ')
        print ('----------------------------------')
        print ('Population size      : {}' .format(population_size))
        print ('Number of generations: {}'.format(generations_number))
        print ('Crossover probability: {}'.format(probability_of_crossover))
        print ('Mutation probability : {}'.format(probability_of_mutation))
        print ('knapsack capacity    : {}'.format(knapsack_capacity))
        print ('Max backpack weight  : {}'.format(max_weight))
        
        
        execute_genetic_algo()

Enter the knapsack capacity:
2
Enter the maximum weight of knapsack:
5
Enter the weight of item1
6
Enter the Value of item1
5
Enter the weight of item2
5
Enter the Value of item2
4
 Generated Parameters             
----------------------------------
Population size      : 87
Number of generations: 193
Crossover probability: 0.4
Mutation probability : 0.3
knapsack capacity    : 2
Max backpack weight  : 5
193

Current generation  : 0
  Best solution so far: 4

Current generation  : 50
  Best solution so far: 4

Current generation  : 100
  Best solution so far: 4

Current generation  : 150
  Best solution so far: 4

--------------------------------
 solution found:

Weight: 5
Value : 4

The Best Chromosome: [0 1]
